In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import re
import os

In [12]:
## Loading data - an excell file :

data1 = pd.ExcelFile('Tranzact2.xlsx')
tranzact_data1 = data1.parse(0)

lamba = len(tranzact_data1)
print(lamba)
tranzact_data1.columns

28922


Index(['Company', 'Supplier', 'Item', 'Category', 'UOM', 'Unnamed: 5',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14',
       'Unnamed: 15', 'Unnamed: 16'],
      dtype='object')

In [13]:
tranzact_data2 = tranzact_data1.iloc[0:lamba, 0:3] ## Remove unnecessary columns

tranzact_data2['Item'] = tranzact_data2['Item'].map(lambda x: x if type(x)!=str else x.lower()) ## make Items in lower case

tranzact_data = tranzact_data2.dropna() ## remove rows with cell value none

tranzact_data = tranzact_data.reset_index(drop=True)
tranzact_data.head()

,Company,Supplier,Item
0,M/S Plastech,Pukhraj Additives LLP,zincoplast zinc
1,M/S Plastech,Pukhraj Additives LLP,zincoplast pyn
2,Modern Engineering & Spring Company,Bijal Steels,"chromium manganese spring steel as rolled, sup..."
3,Modern Engineering & Spring Company,Bijal Steels,"chromium manganese spring steel as rolled, sup..."
4,Modern Engineering & Spring Company,Bijal Steels,"chromium manganese spring steel as rolled, sup..."


In [15]:
## Cleaning the Item Description to create a feature matrix :

pd.options.mode.chained_assignment = None
# print(tranzact_data['Item'][5])

tranzact_data['features'] = tranzact_data['Item'].str.replace(r'\b\d+\b','')  ## remove integers
# print(tranzact_data['features'][5])

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\W',' ')   ## remove puntuations
# print(tranzact_data['features'][5])

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\b\d+\mm\b',' ')   ## remove all 'mm' dimensions only
# print(tranzact_data['features'][5])

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\b\w\b','') ## remove stand alone single letters
# print(tranzact_data['features'][5])

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\s+',' ') ## remove gaps between words to singe gap
# print(tranzact_data['features'][5])


## Removing unneccessary words like measurment units etc.
                                                                        
words = stopwords.words("english")

words.append('mm')
words.append('ft')
words.append('as')
words.append('to')
words.append('nos')

tranzact_data['features']= tranzact_data['features'].apply(lambda x:' '.join([i for i in x.split()
                                                                                if i not in words]).lower())
# print(tranzact_data['features'][5])

for i in range(len(tranzact_data)):
    if tranzact_data['features'][i] == '':
        
        tranzact_data['Supplier'][i] = np.nan   # Remove blank features, if any.
    
        
tranzact_data = tranzact_data.dropna()
tranzact_data = tranzact_data.reset_index(drop=True)


print(len(tranzact_data))

28913


In [16]:
## To check no. of Suppliers & no. of unique features in the data base :

print('Supplier = ', len(tranzact_data.Supplier.value_counts()))
print('Unique Features = ', len(tranzact_data.features.value_counts()))
print('Tranzact_data length = ', len(tranzact_data))
tranzact_data.head()

Supplier =  2337
Unique Features =  16727
Tranzact_data length =  28913


,Company,Supplier,Item,features
0,M/S Plastech,Pukhraj Additives LLP,zincoplast zinc,zincoplast zinc
1,M/S Plastech,Pukhraj Additives LLP,zincoplast pyn,zincoplast pyn
2,Modern Engineering & Spring Company,Bijal Steels,"chromium manganese spring steel as rolled, sup...",chromium manganese spring steel rolled sup9 wi...
3,Modern Engineering & Spring Company,Bijal Steels,"chromium manganese spring steel as rolled, sup...",chromium manganese spring steel rolled sup9 wi...
4,Modern Engineering & Spring Company,Bijal Steels,"chromium manganese spring steel as rolled, sup...",chromium manganese spring steel rolled sup9 wi...


In [7]:
## Above indicates that there are 2337 suppliers supplying 28921 items(as per no. of rows)
## above indicates that there are 16728 unique features out of 28921 items(as per rows)

In [8]:
## Let's reduce no. of Suppliers : criterian - apperaing at least 10 times in the data frame

# tranzact_data_short = tranzact_data.groupby("Supplier").filter(lambda x: len(x) > 10)

# print(len(tranzact_data_short))
# print(len(tranzact_data_short.Supplier.value_counts()))

In [9]:
## Above indicates that there are now 391 suppliers supplying 23649 items(as per no. of rows)

In [19]:
tranzact_data.head()

,Company,Supplier,Item,features
0,M/S Plastech,Pukhraj Additives LLP,zincoplast zinc,zincoplast zinc
1,M/S Plastech,Pukhraj Additives LLP,zincoplast pyn,zincoplast pyn
2,Modern Engineering & Spring Company,Bijal Steels,"chromium manganese spring steel as rolled, sup...",chromium manganese spring steel rolled sup9 wi...
3,Modern Engineering & Spring Company,Bijal Steels,"chromium manganese spring steel as rolled, sup...",chromium manganese spring steel rolled sup9 wi...
4,Modern Engineering & Spring Company,Bijal Steels,"chromium manganese spring steel as rolled, sup...",chromium manganese spring steel rolled sup9 wi...


In [21]:
## Removing duplicate features + Supplier combinations : (To reduce time)

for i in range (len(tranzact_data)):
    
    idx_feature = []
    idx_Supplier = []
    idx_common = []

    idx_feature = tranzact_data.index[tranzact_data['features'] == tranzact_data['features'][i]].tolist()
    idx_Supplier = tranzact_data.index[tranzact_data['Supplier'] == tranzact_data['Supplier'][i]].tolist()
    idx_common = list(set(idx_feature).intersection(idx_Supplier))
    
    if len(idx_common) > 1:
        
        tranzact_data['Supplier'][idx_common[1:]] = np.nan
        
tranzact_data_final = tranzact_data.dropna()
tranzact_data_final = tranzact_data_final.reset_index(drop=True) 


# tranzact_data = tranzact_data.drop(tranzact_data.index[idx_common[j]][1:])
                   

In [22]:
## To cross check no. of Suppliers & no. of unique features in the data base :

print('Supplier = ', len(tranzact_data_final.Supplier.value_counts()))
print('Unique Features = ', len(tranzact_data_final.features.value_counts()))
print('Total no. of entrys = ', len(tranzact_data_final))
tranzact_data_final.tail()

Supplier =  2337
Unique Features =  16727
Total no. of entrys =  19761


,Company,Supplier,Item,features
19756,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,pvc gland pg11,pvc gland pg11
19757,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,pvc gland pg13.5,pvc gland pg13
19758,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,pvc gland m25x1.5,pvc gland m25x1
19759,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 1mm '1',ferrul number
19760,Airbex System Pvt. Ltd,Anant Traders,"7/8"" hard copper pipe 20 gauge 10' long",hard copper pipe gauge long


In [23]:
col = ['Supplier', 'features']
tranzact_data_search = tranzact_data_final[col]   ## This is the file to use for searching


tranzact_data_search['Supplier_id'] = tranzact_data_search['Supplier'].factorize()[0]  ## Indexing & creating Supplier ID

supplier_id_tranzact_data_search = tranzact_data_search[['Supplier', 'Supplier_id']].drop_duplicates(). \
                                                        sort_values('Supplier_id')

## Create two dictionaries :

supplier_to_id = dict(supplier_id_tranzact_data_search.values)
id_to_supplier = dict(supplier_id_tranzact_data_search[['Supplier_id', 'Supplier']].values)

supplier_id_tranzact_data_search.head()



,Supplier,Supplier_id
0,Pukhraj Additives LLP,0
2,Bijal Steels,1
3,Jay Khodiyar Steel & Metals,2
4,Dynachem Engineers,3
6,Sharp Engineering,4


In [24]:
## List of suppliers with created supplier id
print(len(supplier_id_tranzact_data_search))
supplier_id_tranzact_data_search.tail()

2337


,Supplier,Supplier_id
19741,S C Industrial Syndicate,2332
19742,Maxell Heat Exchangers Pvt Ltd,2333
19746,Imerys Minerals Malaysia Sdn Bhd (86796-M),2334
19750,Sigma Roto Lining LLP,2335
19751,Guangdong Yuxing Fire-retardant New Materials ...,2336


In [25]:
tranzact_data_search.tail()

,Supplier,features,Supplier_id
19756,M/S. S. K. Enterprise,pvc gland pg11,312
19757,M/S. S. K. Enterprise,pvc gland pg13,312
19758,M/S. S. K. Enterprise,pvc gland m25x1,312
19759,M/S. S. K. Enterprise,ferrul number,312
19760,Anant Traders,hard copper pipe gauge long,2135


In [26]:
# Digitising Process No. 1

from sklearn.feature_extraction.text import TfidfVectorizer

# create the transform
my_vectorizer = TfidfVectorizer()

# tokenize and build vocab
digital_features = my_vectorizer.fit(tranzact_data_search['features'])
# summarize
# print(my_vectorizer.vocabulary_)

# encode document
my_vector = digital_features.transform(tranzact_data_search['features']).toarray()

In [27]:
## Creating cosine_similarity module :

from math import*
 
def square_rooted(x):
 
    return round(sqrt(sum([a*a for a in x])),3)
 
def cosine_similarity(x,y):
 
    numerator = sum(a*b for a,b in zip(x,y))
    denominator = square_rooted(x)*square_rooted(y)
    return round(numerator/float(denominator),3)

In [28]:
## Inputting text data :
text_data = ['spring steel coil']
text_data1 = pd.Series(text_data)
print(type(text_data1))

<class 'pandas.core.series.Series'>


In [29]:
## Creating text feature

pd.options.mode.chained_assignment = None

text_data_feature = text_data1.str.replace(r'\b\d+\b','')  ## remove integers

text_data_feature = text_data_feature.str.replace(r'\W',' ')   ## remove puntuations

text_data_feature = text_data_feature.str.replace(r'\b\d+\mm\b',' ')   ## remove all 'mm' dimensions only

text_data_feature = text_data_feature.str.replace(r'\b\w\b','') ## remove stand alone single letters

text_data_feature = text_data_feature.str.replace(r'\s+',' ') ## remove gaps between words to singe gap


## Removing unneccessary words like measurment units etc.
                                                                        
words = stopwords.words("english")

words.append('mm')
words.append('ft')
words.append('as')
words.append('to')
words.append('nos')

text_data_feature = text_data_feature.apply(lambda x:' '.join([i for i in x.split()
                                                                                if i not in words]).lower())
print(text_data_feature)

0    spring steel coil
dtype: object


In [32]:
## Creating digitised text vector and calculating cosine similarity with original digitised feature vector :
## To reduce time

merge_text_feature = []
merge_text_feature_digit = []
merge_digit_vector = []
test_digit_vector = []

merge_text_feature = tranzact_data_search['features'].append(pd.Series(text_data_feature))

merge_text_feature = merge_text_feature.reset_index(drop=True)

merge_text_feature_digit = my_vectorizer.fit(merge_text_feature)

# encode document
merge_digit_vector = merge_text_feature_digit.transform(merge_text_feature).toarray()


test_digit_vector = merge_digit_vector[len(merge_digit_vector)-1].reshape((merge_digit_vector.shape)[1],)

print(my_vector[19760].shape)
print(test_digit_vector.shape)

# print(cosine_similarity(my_vector[19760], test_digit_vector))
# print(max(my_vector[19760]))

# c_s = []

# for i in range(len(my_vector)):
    
#    c_s.append(cosine_similarity(my_vector[i], test_digit_vector))

# print(len(c_s))


# print(type(c_s))



(13421,)
(13421,)


In [37]:
# Searching & printing maximum top 5 suppliers :

max_c_s_index = np.argsort(c_s)[-10:][::-1]

for k in range(len(max_c_s_index)):
    
    if tranzact_data_search['Supplier_id'][max_c_s_index[k]] \
                 == tranzact_data_search['Supplier_id'][max_c_s_index[k+1]]:
        
        max_c_s_index = np.delete(max_c_s_index, k)
                
    if k == len(max_c_s_index)-2:
        
        break
        

kount = 0

if c_s[max_c_s_index[0]] < 0.5:
    
    print('There are No suppliers for this item')
    
else:
    
    for i in range(len(max_c_s_index)):
        
        if c_s[max_c_s_index[i]] < 0.5:
            
            print('There are No more Suppliers')
                
            break
            
        print(tranzact_data_search['Supplier'][max_c_s_index[i]])            
        kount = kount+1
            
        if kount == 5 :
            break


Steel N Alloys
CESARE BONETTI INDIA PVT. LTD.
Titan Stainless India Pvt. Ltd.
Max Forge
Sun Metal & Alloys


In [ ]:
def cos_cdist(matrix, vector):
    """
    Compute the cosine distances between each row of matrix and vector.
    """
    v = vector.reshape(1, -1)
    return scipy.spatial.distance.cdist(matrix, v, 'cosine').reshape(-1)

In [ ]:
c_s_1 = cos_cdist(my_vector, text_digit_vector)

In [ ]:
max_c_s_1_index = np.argsort(c_s_1)[-10:][::-1]

for k in range(len(max_c_s_1_index)):
    
    if tranzact_data_search['Supplier_id'][max_c_s_1_index[k]] \
                 == tranzact_data_search['Supplier_id'][max_c_s_1_index[k+1]]:
        
        max_c_s_1_index = np.delete(max_c_s_1_index, k)
                
    if k == len(max_c_s_1_index)-2:
        
        break
        

kount = 0

if c_s_1[max_c_s_1_index[0]] < 0.5:
    
    print('There are No suppliers for this item')
    
else:
    
    for i in range(len(max_c_s_1_index)):
        
        if c_s_1[max_c_s_1_index[i]] < 0.5:
            
            print('There are No more Suppliers')
                
            break
            
        print(tranzact_data_search['Supplier'][max_c_s_1_index[i]])            
        kount = kount+1
            
        if kount == 5 :
            break


In [47]:
from sklearn.metrics.pairwise import cosine_similarity
import scipy

# print(cosine_similarity([[1, 0, -1, 0]], [[-1,-1, 0]]))

# print(my_vector[0])

# cosine_similarity([my_vector[0]], [text_vector])


In [48]:
def cos_cdist(matrix, vector):
    """
    Compute the cosine distances between each row of matrix and vector.
    """
    v = vector.reshape(1, -1)
    return scipy.spatial.distance.cdist(matrix, v, 'cosine').reshape(-1)

In [51]:
## Creating digitised text vector and calculating cosine similarity with original digitised feature vector :
## To reduce time

merge_text_feature = []
merge_text_feature_digit = []
merge_digit_vector = []
text_digit_vector = []

merge_text_feature = tranzact_data_search['features'].append(pd.Series(text_data_feature))

merge_text_feature = merge_text_feature.reset_index(drop=True)

merge_text_feature_digit = my_vectorizer.fit(merge_text_feature)

# encode document
merge_digit_vector = merge_text_feature_digit.transform(merge_text_feature).toarray()

# Next line extracting digitised text vector :

text_digit_vector = merge_digit_vector[len(merge_digit_vector)-1].reshape((merge_digit_vector.shape)[1],)

# Next lines for calculating cosine similarity with original digitised feature vector :

c_s_1 = []

#d = np.tile(b,(3,1))

c_s_1 = cos_cdist(my_vector, text_digit_vector)



